In [77]:
# Import Neccessary Libraries:

import numpy as np
import pandas as pd
import torch
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torch import nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm
import pickle
from torch.optim.lr_scheduler import StepLR
import time
from sklearn.model_selection import train_test_split
from PIL import Image
from itertools import chain
from sklearn import preprocessing

In [78]:
raw_df = pd.read_csv("/content/Data_Entry_2017.csv")


In [79]:
raw_df.head()


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [80]:
raw_df.shape

(112120, 12)

In [81]:
mlb = preprocessing.MultiLabelBinarizer()

In [82]:
f1 = lambda x: x.strip().split("|")

In [83]:
raw_df["Finding Labels"] = raw_df["Finding Labels"].apply(f1)

In [84]:
raw_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,[Cardiomegaly],0,1,58,M,PA,2682,2749,0.143,0.143,NaN
1,00000001_001.png,"[Cardiomegaly, Emphysema]",1,1,58,M,PA,2894,2729,0.143,0.143,NaN
2,00000001_002.png,"[Cardiomegaly, Effusion]",2,1,58,M,PA,2500,2048,0.168,0.168,NaN
3,00000002_000.png,[No Finding],0,2,81,M,PA,2500,2048,0.171,0.171,NaN
4,00000003_000.png,[Hernia],0,3,81,F,PA,2582,2991,0.143,0.143,NaN


In [85]:
mlb.fit(raw_df["Finding Labels"])

MultiLabelBinarizer()

In [86]:
temp = mlb.transform(raw_df["Finding Labels"])

In [87]:
raw_df[mlb.classes_] = temp

In [88]:
raw_df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000001_000.png,[Cardiomegaly],0,1,58,M,PA,2682,2749,0.143,0.143,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,"[Cardiomegaly, Emphysema]",1,1,58,M,PA,2894,2729,0.143,0.143,NaN,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,00000001_002.png,"[Cardiomegaly, Effusion]",2,1,58,M,PA,2500,2048,0.168,0.168,NaN,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,00000002_000.png,[No Finding],0,2,81,M,PA,2500,2048,0.171,0.171,NaN,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,00000003_000.png,[Hernia],0,3,81,F,PA,2582,2991,0.143,0.143,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [89]:
raw_df2 = raw_df.drop(raw_df.loc[raw_df["No Finding"] == 1].sample(frac=0.95).index)

In [90]:
# raw_df.loc[raw_df["No Finding"] == 1].sample(frac=0.95)

In [91]:
raw_df2

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax
0,00000001_000.png,[Cardiomegaly],0,1,58,M,PA,2682,2749,0.143000,0.143000,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,00000001_001.png,"[Cardiomegaly, Emphysema]",1,1,58,M,PA,2894,2729,0.143000,0.143000,NaN,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
2,00000001_002.png,"[Cardiomegaly, Effusion]",2,1,58,M,PA,2500,2048,0.168000,0.168000,NaN,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,00000003_000.png,[Hernia],0,3,81,F,PA,2582,2991,0.143000,0.143000,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,00000003_001.png,[Hernia],1,3,74,F,PA,2500,2048,0.168000,0.168000,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112097,00030786_007.png,"[Consolidation, Pleural_Thickening]",7,30786,61,F,AP,3056,2544,0.139000,0.139000,NaN,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
112100,00030789_000.png,[Infiltration],0,30789,52,F,PA,2021,2021,0.194311,0.194311,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
112106,00030793_000.png,"[Mass, Nodule]",0,30793,58,F,PA,2021,2021,0.194311,0.194311,NaN,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
112108,00030795_000.png,[Pleural_Thickening],0,30795,53,F,PA,2021,2021,0.194311,0.194311,NaN,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [92]:
df = raw_df2.copy()

In [93]:
# all_xray_df = pd.read_csv("/content/final_df_v2.csv")
all_labels = mlb.classes_
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))

All Labels (15): ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']


In [94]:
MIN_CASES = 1000

all_labels = [c_label for c_label in all_labels if df[c_label].sum()>MIN_CASES]
print('Clean Labels ({})'.format(len(all_labels)), 
      [(c_label,int(df[c_label].sum())) for c_label in all_labels])

Clean Labels (14) [('Atelectasis', 11559), ('Cardiomegaly', 2776), ('Consolidation', 4667), ('Edema', 2303), ('Effusion', 13317), ('Emphysema', 2516), ('Fibrosis', 1686), ('Infiltration', 19894), ('Mass', 5782), ('No Finding', 3018), ('Nodule', 6331), ('Pleural_Thickening', 3385), ('Pneumonia', 1431), ('Pneumothorax', 5302)]


In [95]:
df = df.drop(df.loc[df["Hernia"] == 1].index)
df['disease_vec'] = df.apply(lambda x: [x[all_labels].values], 1).map(lambda x: x[0])


In [96]:
df.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax,disease_vec
0,00000001_000.png,[Cardiomegaly],0,1,58,M,PA,2682,2749,0.143,0.143,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,00000001_001.png,"[Cardiomegaly, Emphysema]",1,1,58,M,PA,2894,2729,0.143,0.143,NaN,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
2,00000001_002.png,"[Cardiomegaly, Effusion]",2,1,58,M,PA,2500,2048,0.168,0.168,NaN,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
12,00000004_000.png,"[Mass, Nodule]",0,4,82,M,AP,2500,2048,0.168,0.168,NaN,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]"
19,00000005_006.png,[Infiltration],6,5,70,F,PA,2992,2991,0.143,0.143,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


In [97]:
df["Image Index"].nunique()

54550

In [98]:
len(mlb.classes_)

15

In [99]:
len(df["disease_vec"][0])

14

In [100]:
old_df = pd.read_csv("/content/final_df_v2.csv")

In [103]:
df[~df["Image Index"].isin(old_df["Image Index"].tolist())]

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11,Atelectasis,Cardiomegaly,Consolidation,Edema,Effusion,Emphysema,Fibrosis,Hernia,Infiltration,Mass,No Finding,Nodule,Pleural_Thickening,Pneumonia,Pneumothorax,disease_vec
0,00000001_000.png,[Cardiomegaly],0,1,58,M,PA,2682,2749,0.143000,0.143000,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,00000001_002.png,"[Cardiomegaly, Effusion]",2,1,58,M,PA,2500,2048,0.168000,0.168000,NaN,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
12,00000004_000.png,"[Mass, Nodule]",0,4,82,M,AP,2500,2048,0.168000,0.168000,NaN,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]"
19,00000005_006.png,[Infiltration],6,5,70,F,PA,2992,2991,0.143000,0.143000,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
20,00000005_007.png,"[Effusion, Infiltration]",7,5,70,F,PA,2566,2681,0.143000,0.143000,NaN,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112093,00030786_003.png,"[Cardiomegaly, Effusion, Emphysema, Pneumothorax]",3,30786,61,F,AP,3056,2544,0.139000,0.139000,NaN,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,"[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1]"
112096,00030786_006.png,[Consolidation],6,30786,61,F,AP,3056,2544,0.139000,0.139000,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
112097,00030786_007.png,"[Consolidation, Pleural_Thickening]",7,30786,61,F,AP,3056,2544,0.139000,0.139000,NaN,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
112100,00030789_000.png,[Infiltration],0,30789,52,F,PA,2021,2021,0.194311,0.194311,NaN,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


In [106]:
[(i, df[i].sum()) for i in mlb.classes_]

[('Atelectasis', 11519),
 ('Cardiomegaly', 2769),
 ('Consolidation', 4663),
 ('Edema', 2300),
 ('Effusion', 13296),
 ('Emphysema', 2512),
 ('Fibrosis', 1678),
 ('Hernia', 0),
 ('Infiltration', 19861),
 ('Mass', 5757),
 ('No Finding', 3018),
 ('Nodule', 6321),
 ('Pleural_Thickening', 3377),
 ('Pneumonia', 1428),
 ('Pneumothorax', 5293)]

In [107]:
x = pd.read_csv("/content/final_df_v2.csv")

In [109]:
[(i, x[i].sum()*2) for i in mlb.classes_]

[('Atelectasis', 11048.0),
 ('Cardiomegaly', 2614.0),
 ('Consolidation', 4720.0),
 ('Edema', 2434.0),
 ('Effusion', 13020.0),
 ('Emphysema', 2468.0),
 ('Fibrosis', 1504.0),
 ('Hernia', 202.0),
 ('Infiltration', 17236.0),
 ('Mass', 5552.0),
 ('No Finding', 1822.0),
 ('Nodule', 5830.0),
 ('Pleural_Thickening', 3362.0),
 ('Pneumonia', 1484.0),
 ('Pneumothorax', 4790.0)]

In [110]:
df.to_csv("/content/final_df_v3.csv", index = False)